In this notebook I will build a model to detect and recognize American Manual Alphabet (AMA). A model that can understand sign language can help improve the lives of many people and recognizing the Alphabet is a good start for it.
I will use the Xception model that trained on ImageNet dataset and it will demonstrate the power of Transfer Learning. Xception is a deep convolutional neural network architecture developed by Google researchers. It's an efficient architecture that relies on two main points:
- Depthwise Separable Convolution.
- Shortcuts between Convolution blocks as in ResNet.

Xception model got an extraordinary results on ImageNet dataset with less then 20% parameters than VGG16. With my Xception based model I got an accuracy result of allmost 100% on the validation data!


In [ ]:
import os
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Preprocessing:
The dataset contains 41 different signs represented as folders, with 1600 images for each sign - 26 alphabets, 10 digits and 5 extra symbols which are used to perform ancillary operations. For this work, I didn't use the digits because some of them are identical to letters (for example 0 and o, 5 and z) and the only way to distinguish them is by context (not in scope of this work).

In [ ]:
PATH = r'../input/hand-sign-recognition/original_images/original_images'

classes_wo_digits = os.listdir(PATH)

classes_wo_digits.sort()

classes_wo_digits = classes_wo_digits[10:]

print(classes_wo_digits)

I split the data to "train" (80%) and "validation" (20%) using Keras's ImageDataGenerator:

In [ ]:
datagen = keras.preprocessing.image.ImageDataGenerator(
                                            rescale=1./255, 
                                            validation_split=0.2
                                                       )

train_generator = datagen.flow_from_directory(
                                        PATH,
                                        target_size=(256, 256),
                                        seed=51,
                                        subset='training', 
                                        classes=classes_wo_digits
                                              )

validation_generator = datagen.flow_from_directory(
                                        PATH,
                                        target_size=(256, 256), 
                                        seed=51,
                                        subset='validation', 
                                        shuffle=False, 
                                        classes=classes_wo_digits
                                                   )

Here is an example of the first batch:

In [ ]:
fig, axes = plt.subplots(4, 8, sharex=True, figsize=(10, 10))

plt.subplots_adjust(bottom=0.1, left=0.01, right=0.99,
                    top=0.9, hspace=0.35)

for i in range(4):
    for j in range(8):
        plt.gray()
        axes[i, j].imshow(train_generator[0][0][8*i + j])
        axes[i, j].set_title(
          classes_wo_digits[
              train_generator[0][1][8*i + j].argmax()
                            ],
          weight='bold', size=16)
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])

plt.show()

# Building The Model:
I imported the Xception model from keras with weights pre-trained on ImageNet. I didn't use the original top of the model and instead added 3 fully connected (FC) layers (the last one with units as the number of classes). The only trainable weights are the ones of the FC layers.

In [ ]:
xception = Xception(include_top=False, input_shape=(256, 256, 3))

x = xception.output
x = layers.GlobalMaxPooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(31, activation='softmax')(x)

model = Model(xception.input, output)

# Freezing all the Imported Layers
for layers in xception.layers:
    layers.trainable = False
    
model.compile(optimizer='adam', loss="categorical_crossentropy",
              metrics=["accuracy"])

earlystop = EarlyStopping(monitor='val_accuracy', patience=20,
                          verbose=1)

# Training:

In [ ]:
hist = model.fit(train_generator, 
                 validation_data=validation_generator,
                 steps_per_epoch=100, validation_steps=10,
                 epochs=50, callbacks=[earlystop])

**The model finished training after only half of the epochs!**

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("Model Metrics", weight='bold', size=18)
plt.ylabel("Accuracy / Loss", size=14)
plt.xlabel("Epoch", size=14)
plt.legend(["Training Accuracy", "Validation Accuracy",
            "Training Loss", "Validation Loss"])
plt.show()

# Evaluating:

In [ ]:
result = model.evaluate(validation_generator, steps=280)

**The model predict correctly allmost 100% of the valdiation data!**